In [1]:
from crewai_tools import ScrapeWebsiteTool, FileWriterTool, TXTSearchTool
import requests
from crewai import Agent, Task, Crew, Process
import os
from crewai_tools import TXTSearchTool

# Hello World

In [ ]:
# Initialize the tool, potentially passing the session
scrape_tool = ScrapeWebsiteTool(website_url='https://en.wikipedia.org/wiki/Artificial_intelligence')

# Extract the text
text = scrape_tool.run()
print(text[:100])

In [3]:
# Initialize the tool
file_writer_tool = FileWriterTool()

# Write content to a file in a specified directory
result = file_writer_tool._run(filename='ai.txt', content = text, directory = '', overwrite=True)
print(result)

Content successfully written to ai.txt


In [5]:
os.environ['OPENAI_API_KEY'] = 'your_openai_key'

# Initialize the tool with a specific text file, so the agent can search within the given text file's content
# uses chromadb to chunk and vectorize data
search_tool = TXTSearchTool(txt='ai.txt')

In [6]:
from crewai import Agent, Task, Crew

data_analyst = Agent(
    role='Educator',
    goal=f'Based on the context provided, answer the question - What is Natural Language Processing?',
    backstory='You are a data expert',
    verbose=True,
    allow_delegation=False,
    tools=[search_tool]
)

test_task = Task(
    description="Understand the topic of Natural Language Processing and give the correct response",
    agent=data_analyst,
    expected_output='Give a correct response'
)

crew = Crew(
    tasks=[test_task],
    process=Process.sequential,
)

output = crew.kickoff()

# Agent: Educator
## Task: Understand the topic of Natural Language Processing and give the correct response


# Agent: Educator
## Thought: I need to gather detailed information about Natural Language Processing to provide a complete response.
## Using tool: Search a txt's content
## Tool Input: 
"{\"search_query\": \"Natural Language Processing\"}"
## Tool Output: 
Relevant Content:
Natural language processing (NLP)[50] allows programs to read, write and communicate in human languages such as English. Specific problems include speech recognition, speech synthesis, machine translation, information extraction, information retrieval and question answering.[51]
Early work, based on Noam Chomsky's generative grammar and semantic networks, had difficulty with word-sense disambiguation[f] unless restricted to small domains called "micro-worlds" (due to the common sense knowledge problem[29]). Margaret Masterman believed that it was meaning and not grammar that was the key to understanding l

In [7]:
print(output.raw)

Natural language processing (NLP) allows programs to read, write and communicate in human languages such as English. Specific problems include speech recognition, speech synthesis, machine translation, information extraction, information retrieval, and question answering. Early work, based on Noam Chomsky's generative grammar and semantic networks, faced challenges with word-sense disambiguation unless restricted to small domains called "micro-worlds." Margaret Masterman believed that it was meaning, not grammar, that was key to understanding languages, suggesting that thesauri, not dictionaries, should form the basis of computational language structure. 

Modern deep learning techniques for NLP include word embedding (representing words, typically as vectors encoding their meaning), transformers (deep learning architecture using an attention mechanism), and others. In 2019, generative pre-trained transformer (or "GPT") language models began to generate coherent text, and by 2023, thes

In [8]:
from langchain_openai import ChatOpenAI

test_task = Task(
    description="Understand the topic of Natural Language Processing and give the correct response",
    # agent=data_analyst,  # commenting out the agent now, letting the manager decide
    expected_output='Give a correct response'
)
calculator_agent = Agent(
    role='Calculator',
    goal=f'You calculate things',
    backstory='You love math',
    verbose=True,
    allow_delegation=False,
    tools=[]
)

math_task = Task(
    description="Tell me what 123 * 34 is",
    expected_output='Calculate this'
)


crew = Crew(
    tasks=[math_task, test_task],
    agents=[calculator_agent, data_analyst],
    process=Process.hierarchical,
    manager_llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7),
    verbose=True
)

output = crew.kickoff()

2024-10-27 08:09:51,215 - 8440492608 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Crew Manager
## Task: Tell me what 123 * 34 is
# Agent: Calculator
## Task: Calculate


# Agent: Calculator
## Final Answer: 
To calculate the product of 123 and 34, we perform the multiplication as follows:

123  
×  34  
______  
=  492  (this is 123 multiplied by 4, the units place of 34)  
+ 3690  (this is 123 multiplied by 3, the tens place of 34, shifted one position to the left)  
______  
= 4182  

Therefore, when you multiply 123 by 34, the final answer is 4182.




# Agent: Crew Manager
## Thought: You should delegate the task of calculating 123 * 34 to the Calculator coworker. They specialize in mathematical calculations and can provide the exact result we need.
## Using tool: Delegate work to coworker
## Tool Input: 
"{\"task\": \"Calculate\", \"context\": \"Multiply 123 by 34\", \"coworker\": \"Calculator\"}"
## Tool Output: 
To calculate the product of 123 and 34, we perform the multiplication as follows:

123  
×  34  
______  
=  492  (this is 123 multiplied by

In [9]:
for output in output.tasks_output:
    print(output)
    print('----')

123 multiplied by 34 is equal to 4182.
----
Hence, the result of 123 multiplied by 34 is 4182.
----


In [10]:
output.raw

'Hence, the result of 123 multiplied by 34 is 4182.'

# LangChain + CrewAI

In [12]:
import os

from langchain_community.tools import DuckDuckGoSearchRun  # Initialize the tool
search_ddg_tool = DuckDuckGoSearchRun()

In [13]:
search_ddg_tool.run('capital of Turkey')

'Ankara is the national capital of Turkey, located in the northwestern part of the country. It has a rich and diverse history, dating back to the Stone Age, and is home to many cultural and historical attractions, such as the Atatürk Mausoleum and the Museum of Anatolian Civilizations. Turkey is a transcontinental country in Eurasia and the Middle East, with Ankara as its capital and Istanbul as its largest city. Learn about its geography, history, culture, politics, economy and more from this comprehensive Wikipedia article. Istanbul is the largest city and principal seaport of Turkey, and the former capital of both the Byzantine and Ottoman empires. It stands on a triangular peninsula between Europe and Asia, and has a rich cultural and historical heritage, as well as a diverse geography and climate. Learn about Turkey, a country that occupies a unique geographic position, lying partly in Asia and partly in Europe. The capital of Turkey is Ankara, and its largest city and seaport is 

In [ ]:
#
researcher = Agent(
  role='Senior Research Analyst',
  goal='Uncover cutting-edge developments in AI and data science',
  backstory="""You work at a leading tech think tank.
  Your expertise lies in identifying emerging trends.
  You have a knack for dissecting complex data and presenting actionable insights.""",
  verbose=True,
  allow_delegation=False,
  tools=[search_ddg_tool]
)

writer = Agent(
  role='Tech Content Strategist',
  goal='Craft compelling content on tech advancements',
  backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
  verbose=True,
  allow_delegation=True
)

In [ ]:
# Create tasks for your agents
task1 = Task(
  description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.""",
  expected_output="Full analysis report in bullet points",
  agent=researcher
)

task2 = Task(
  description="""Using the insights provided, develop an engaging blog
  post that highlights the most significant AI advancements.
  Your post should be informative yet accessible, catering to a tech-savvy audience.
  Make it sound cool, avoid complex words so it doesn't sound like AI.""",
  expected_output="Full blog post of at least 4 paragraphs",
  agent=writer
)

In [ ]:
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[researcher, writer],
  tasks=[task1, task2],
  process=Process.sequential,
)

In [ ]:
# writes tries to delegate to research analyst. WRONG!
result = crew.kickoff()

print("-----------------------------")
print(result)


# Agent: Senior Research Analyst
## Task: Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.


# Agent: Senior Research Analyst
## Thought: I need to gather information on the latest advancements in AI for the year 2024, focusing on key trends and breakthrough technologies. I will use the available tool to search for relevant current information.
## Using tool: duckduckgo_search
## Tool Input: 
"{\"query\": \"latest advancements in AI 2024 trends technologies breakthroughs\"}"
## Tool Output: 
In 2024, generative AI might actually become useful for the regular, non-tech person, and we are going to see more people tinkering with a million little AI models. State-of-the-art AI models ... When OpenAI launched a free web app called ChatGPT in November 2022, nobody knew what was coming. But that low-key release changed everything. By January, ChatGPT had become the fastest-growing ... H

KeyboardInterrupt: 

In [ ]:
# from langchain_openai import ChatOpenAI

researcher = Agent(
  role='Senior Research Analyst',
  goal='Uncover cutting-edge developments in AI and data science',
  backstory="""You work at a leading tech think tank.
  Your expertise lies in identifying emerging trends.
  You have a knack for dissecting complex data and presenting actionable insights.""",
  verbose=True,
  allow_delegation=False,
  tools=[search_ddg_tool],
#   llm=ChatOpenAI(model_name="gpt-4o-mini", temperature=0.4)
)

writer = Agent(
  role='Tech Content Strategist',
  goal='Craft compelling content on tech advancements',
  backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
  verbose=True,
  allow_delegation=False
)

In [ ]:
# Create tasks for your agents
task1 = Task(
  description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.""",
  expected_output="Full analysis report in bullet points",
  agent=researcher
)

task2 = Task(
  description="""Using the insights provided, develop an engaging blog
  post that highlights the most significant AI advancements.
  Your post should be informative yet accessible, catering to a tech-savvy audience.
  Make it sound cool, avoid complex words so it doesn't sound like AI.""",
  expected_output="Full blog post of at least 4 paragraphs",
  agent=writer
)

In [ ]:
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[researcher, writer],
  tasks=[task1, task2],
  process=Process.sequential,
)
result = crew.kickoff()
print("-----------------------------")
print(result)

# Agent: Senior Research Analyst
## Task: Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.


# Agent: Senior Research Analyst
## Thought: I need to search for the latest advancements in AI in 2024 to gather comprehensive insights about key trends, breakthrough technologies, and their potential industry impacts.
## Using tool: duckduckgo_search
## Tool Input: 
"{\"name\": \"duckduckgo_search\", \"description\": \"A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.\", \"args_schema\": {}, \"return_direct\": false, \"verbose\": false, \"tags\": null, \"metadata\": null, \"handle_tool_error\": false, \"handle_validation_error\": false, \"response_format\": \"content\", \"api_wrapper\": {}}"
## Tool Output: 
Error: the Action Input is not a valid key, value dictionary.


# Agent: Senior Research Analyst

In [ ]:
task1.output.raw

'- **Expansion of Generative AI Usage**:\n  - In 2024, 65% of organizations are regularly using generative AI, a significant increase from previous years.\n  - The focus has shifted from experimenting with generative AI to deriving tangible business value from it.\n\n- **Growth Rate Projections**:\n  - AI is anticipated to grow at an annual rate of 36.6% from 2023 to 2030, indicating strong market demand and potential for innovation.\n\n- **Accessibility to Non-Tech Users**:\n  - Generative AI is becoming more user-friendly, enabling regular users to engage with and utilize various AI models in their daily tasks.\n\n- **Improved Autonomous Chatbots**:\n  - Autonomous chatbots are evolving with smarter algorithms, enhancing customer service capabilities, and reducing response times.\n\n- **AI Tinkering and Model Development**:\n  - An increasing number of users are experimenting with developing or utilizing a multitude of small-scale AI models to meet specific needs.\n\n- **Diverse Appl

In [ ]:
task2.output.raw

"**The Future is Now: 2024’s Dazzling AI Advancements**\n\nWelcome to 2024, where artificial intelligence is not just another buzzword but a dynamic force reshaping the business landscape. In a world where 65% of organizations are now actively harnessing the power of generative AI, we're witnessing a monumental shift. Gone are the days of merely experimenting with AI; companies are focusing on extracting real-world value from it. From automating mundane tasks to generating insightful data-driven decisions, generative AI is becoming the backbone of innovation, driving efficiency, and giving businesses a competitive edge like never before.\n\nWhat's even more exhilarating is the staggering growth rate projected for AI. Anticipated to rise at an amazing annual rate of 36.6% from 2023 to 2030, AI is not just a passing trend; it’s a burgeoning market demand. With increasing accessibility, even those without a tech background are diving into AI. User-friendly platforms now allow everyone to 

# Building a Search Agent with a Custom Tool

In [17]:
# use langhcain chatllm
from langchain_openai import ChatOpenAI

researcher = Agent(
  role='Senior Research Analyst',
  goal='Uncover cutting-edge developments in AI and data science',
  backstory="""You work at a leading tech think tank.
  Your expertise lies in identifying emerging trends.
  You have a knack for dissecting complex data and presenting actionable insights.""",
  verbose=True,
  allow_delegation=False,
  tools=[search_ddg_tool],
  llm=ChatOpenAI(model_name="gpt-4o-mini", temperature=0.4)
)

In [ ]:
os.environ["SERP_API_KEY"] = 'your_serp_key'

In [22]:
import json
import os
import requests
from crewai import Agent, Task
from langchain.tools import tool
from serpapi import GoogleSearch
from crewai_tools import BaseTool

class SearchTool(BaseTool):
    name: str = "Google Search"
    description: str = """Useful to search the internet about a given topic and return relevant results."""

    def _run(self, query: str, top_k: int = 3) -> str:
        params: dict = {
            "engine": "google",
            "google_domain": "google.com",
            "gl": "us",
            "hl": "en",
            "q": query,
            "api_key": os.environ["SERP_API_KEY"],
        }

        search = GoogleSearch(params)
        response = search.get_dict()
        # check if there is an organic results key, don't include sponsor results
        if 'organic_results' not in response:
            return "Sorry, I couldn't find anything about that, there could be an error with you serper api key."
        else:
            results = response['organic_results']
            string = []
            for result in results[:top_k]:
                try:
                    string.append('\n'.join([
                        f"Title: {result['title']}", f"Link: {result['link']}",
                        f"Snippet: {result['snippet']}", "\n-----------------"
                    ]))
                except KeyError:
                    next

            return '\n'.join(string)

In [23]:
SearchTool().description # where I specifically describe how to use the tool

"Google Search(query: 'string', top_k: 'integer') - Useful to search the internet about a given topic and return relevant results. "

In [24]:
SearchTool().run(query="Who is Sinan Ozdemir?")

Using Tool: Google Search


"Title: Sinan Ozdemir - LoopGenius\nLink: https://www.linkedin.com/in/sinan-ozdemir\nSnippet: Specialized in areas including time management as well as mathematics and computer science logistics.\n\n-----------------\nTitle: Sinan Ozdemir\nLink: https://sinanozdemir.ai/\nSnippet: Hi! My name is Sinan Ozdemir. I'm an AI expert, consultant, author, and founder. The author of Quick Start Guide to LLMs (companion Github) and the AI ...\n\n-----------------\nTitle: Sinan Ozdemir - Instructor\nLink: https://www.datacamp.com/portfolio/profoz\nSnippet: Sinan Ozdemir is a Data Scientist, Entrepreneur, Teacher, and Author. He is the founder of LoopGenius, a company that helps people get their first 100 customers ...\n\n-----------------"

In [27]:
researcher_agent = Agent(
    role='Researcher',
    goal='Look up information on the internet and scrape the websites for relevant information',
    backstory="""You have a knack for dissecting complex data and presenting actionable insights.""",
    tools=[
        SearchTool()
    ],
    verbose=True
)
email_agent = Agent(
    role='Emailer',
    goal='Write short, warm emails to people',
    backstory="""You are an assistant.""",
    tools=[],
    verbose=True
)

sample_lookup_task = Task(
  description="""Lookup Sinan Ozdemir""",
  expected_output="A lookup report of a query",
  agent=researcher_agent
)

email_task = Task(
  description="""Write an Sinan Ozdemir asking him to get coffee. Personalize it""",
  expected_output="A ashort warm email",
  agent=email_agent
)

crew = Crew(
    tasks=[sample_lookup_task, email_task],
    agents=[researcher_agent],
    process=Process.sequential,
    verbose=True
)

result = crew.kickoff()

2024-10-27 08:14:28,447 - 8440492608 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Researcher
## Task: Lookup Sinan Ozdemir


# Agent: Researcher
## Thought: I need to find comprehensive information about Sinan Ozdemir to create a lookup report. I'll start by conducting a search to gather relevant data.
## Using tool: Google Search
## Tool Input: 
"{\"query\": \"Sinan Ozdemir\", \"top_k\": 3}"
## Tool Output: 
Title: Sinan Ozdemir - LoopGenius
Link: https://www.linkedin.com/in/sinan-ozdemir
Snippet: Specialized in areas including time management as well as mathematics and computer science logistics.

-----------------
Title: Sinan Ozdemir
Link: https://sinanozdemir.ai/
Snippet: Hi! My name is Sinan Ozdemir. I'm an AI expert, consultant, author, and founder. The author of Quick Start Guide to LLMs (companion Github) and the AI ...

-----------------
Title: Prof_OZ - Sinan Ozdemir
Link: https://twitter.com/prof_oz
Snippet: NLP + Gen AI Expert / LLM whisperer. AI Author. Founder @Aikylie (acquired). Contributor on @Forbes. Fellow @YCombinator. San Francisco, CA

In [28]:
print(result)

Subject: Coffee Catch-Up?  

Hi Sinan,  

I hope this message finds you well! I’ve been following your remarkable work in AI and came across your "Quick Start Guide to LLMs" on GitHub — it’s genuinely inspiring!  

I’d love the opportunity to grab a coffee together sometime. It would be fantastic to chat about your experiences with Aikylie and your insights on the future of natural language processing.  

Looking forward to hearing from you!  

Warm regards,  
[Your Name]  
[Your Contact Information]


## Trip Advisor

In [29]:
from crewai import Agent

class TripAgents():

  def local_expert(self):
    return Agent(
        role='Local Expert at this city',
        goal='Provide 2-3 insights about the selected city',
        backstory="""A knowledgeable local guide with extensive information
        about the city, it's attractions and customs""",
        tools=[
            SearchTool(),
            scrape_tool
        ],
        verbose=True)

  def travel_concierge(self):
    return Agent(
        role='Amazing Travel Concierge',
        goal="""Create the most amazing travel itineraries with budget and
        packing suggestions for the city""",
        backstory="""Specialist in travel planning and logistics with
        decades of experience""",
        tools=[],
        verbose=True)

In [30]:
from crewai import Task
from textwrap import dedent
from datetime import date


class TripTasks():

  def gather_task(self, agent, origin, city, interests, range):
    return Task(description= dedent(f"""
        I want you to create a comprehensive city guide.
        This guide should provide a thorough overview of what
        the city has to offer, including hidden gems, cultural
        hotspots, and must-visit landmarks.

        Trip Date: {range}
        Traveling from: {origin}
        Traveling to: {city}
        Traveler Interests: {interests}
        """),

            expected_output=dedent(f"""
        The final answer must be a comprehensive city guide,
        rich in cultural insights and practical tips,
        tailored to enhance the travel experience.

        """),
                agent=agent)

  def plan_task(self, agent, origin, city, interests, range):
    return Task(description=dedent(f"""
        Expand this guide into a a full 7-day travel
        itinerary with detailed per-day plans, including
        weather forecasts, places to eat, packing suggestions,
        and a budget breakdown.

        You MUST suggest actual places to visit, actual hotels
        to stay and actual restaurants to go to.

        Trip Date: {range}
        Traveling from: {origin}
        Traveling to: {city}
        Traveler Interests: {interests}
      """),
            expected_output=dedent(f"""
        Your final answer MUST be a complete expanded travel plan,
        formatted as markdown, encompassing a daily schedule,
        anticipated weather conditions, recommended clothing and
        items to pack, and a detailed budget, ensuring THE BEST
        TRIP EVER, Be specific and give it a reason why you picked
        # up each place, what make them special!
        """),
                agent=agent)

In [31]:
class TripCrew:

  def __init__(self, origin, city, date_range, interests):
    self.city = city
    self.origin = origin
    self.interests = interests
    self.date_range = date_range

  def run(self):
    agents = TripAgents()
    tasks = TripTasks()

    gather_task = tasks.gather_task(
      agents.local_expert(),
      self.origin,
      self.city,
      self.interests,
      self.date_range
    )
    plan_task = tasks.plan_task(
      agents.travel_concierge(),
      self.origin,
      self.city,
      self.interests,
      self.date_range
    )

    # Initiate Crew
    crew = Crew(
      tasks=[gather_task, plan_task],
      process=Process.sequential,
      verbose=True
    )

    result = crew.kickoff()
    return result

In [32]:
trip_crew = TripCrew('SF', "Paris", 'January 2025', 'art')
result = trip_crew.run()

2024-10-27 08:14:52,393 - 8440492608 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Local Expert at this city
## Task: 
I want you to create a comprehensive city guide.
This guide should provide a thorough overview of what
the city has to offer, including hidden gems, cultural
hotspots, and must-visit landmarks.

Trip Date: January 2025
Traveling from: SF
Traveling to: Paris
Traveler Interests: art



# Agent: Local Expert at this city
## Thought: I need to gather detailed insights about Paris, focusing on its art scene, cultural hotspots, and must-visit landmarks to create a comprehensive city guide.
## Using tool: Google Search
## Tool Input: 
"{\"query\": \"Paris city guide art attractions cultural hotspots must-visit landmarks January 2025\", \"top_k\": 5}"
## Tool Output: 
Title: Paris Events January 2025 | Things to Do
Link: https://www.parisdiscoveryguide.com/paris-events-january.html
Snippet: Saint-Jacques Scallop Festival in Montmartre · Paris Museum Exhibitions for January 2025 · Recommended January Day Trip from Paris: Champagne Tour & Tastings!

-

In [33]:
print(result)

# 7-Day Travel Itinerary for Paris (January 2025) - Art Lovers' Dream

## Day 1 - Arrival (January 5, 2025)
- **Weather**: Expect chilly weather around 5°C (41°F) with possible light rain.
- **Accommodation**: Check-in at Hôtel des Grands Boulevards ($150/night).
- **Activities**:
  - Arrive at Charles de Gaulle Airport, take a taxi/RER to your hotel.
  - Explore the vibrant neighborhood of **Montmartre**, visiting Sacré-Cœur Basilica for panoramic views.
- **Dinner**: Eat at Le Consulat for a cozy French meal ($30). 
- **Packing Suggestion**: Warm layers, waterproof jacket, comfortable walking shoes.

## Day 2 - The Louvre & Le Marais (January 6, 2025)
- **Weather**: Mostly cloudy, about 6°C (43°F).
- **Breakfast**: Croissant and coffee at **Du Pain et des Idées** ($10).
- **Activities**:
  - **Morning**: Visit the **Louvre Museum** (book tickets online - $20). Keys artists: Mona Lisa, The Venus de Milo.
  - **Lunch**: Casual dining at **Café Marly** with views over the Louvre gardens

In [37]:
print("## Welcome to Trip Planner Crew")
print('-——————————')
location = input(
dedent("""
  From where will you be traveling from?
"""))
city = input(
dedent("""
  Where do you want to go?
"""))
date_range = input(
dedent("""
  What is the date range you are interested in traveling?
"""))
interests = input(
dedent("""
  What are some of your high level interests and hobbies?
"""))

trip_crew = TripCrew(location, city, date_range, interests)
result = trip_crew.run()

## Welcome to Trip Planner Crew
-——————————



From where will you be traveling from?
 Trenton, NJ

Where do you want to go?
 Cincinatti

What is the date range you are interested in traveling?
 march 2025

What are some of your high level interests and hobbies?
 kid friendly stuff and science musems


2024-10-27 08:17:05,822 - 8440492608 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Local Expert at this city
## Task: 
I want you to create a comprehensive city guide.
This guide should provide a thorough overview of what
the city has to offer, including hidden gems, cultural
hotspots, and must-visit landmarks.

Trip Date: march 2025
Traveling from: Trenton, NJ
Traveling to: Cincinatti
Traveler Interests: kid friendly stuff and science musems



# Agent: Local Expert at this city
## Thought: I need to gather detailed information about Cincinnati, including attractions, activities, and specific places of interest for families and science enthusiasts.
## Using tool: Google Search
## Tool Input: 
"{\"query\": \"family friendly attractions Cincinnati science museums\", \"top_k\": 5}"
## Tool Output: 
Title: The Children's Museum
Link: https://www.cincymuseum.org/childrensmuseum/
Snippet: The Children's Museum is designed for children ages 0-10, but families of all ages can explore together. We encourage kids to climb, crawl, explore and learn.

-----------------

In [38]:
print("\n\n########################")
print("## Here is you Trip Plan")
print("########################\n")
print(result)



########################
## Here is you Trip Plan
########################

# 7-Day Cincinnati Travel Itinerary (March 2025)

## Overview
Cincinnati is a delightful city for families, rich in learning experiences, especially focused on science. This 7-day itinerary outlines daily plans filled with kid-friendly attractions, detailed budgets, packing tips, and designated meals at delicious local restaurants. The weather in March averages around 50°F (10°C) during the day but can be cooler in the evenings, so come prepared!

---

### Day 1: Arrival in Cincinnati
- **Weather Forecast:** 50°F, partly cloudy
- **Packing Suggestions:** Light jacket, casual clothes, comfortable walking shoes
- **Budget Breakdown:**
  - **Hotel:** Hampton Inn & Suites Cincinnati – Downtown ($200/night)
  - **Meals:** Breakfast (included), Lunch: Skyline Chili - $20, Dinner: Montgomery Inn - $60
  - **Transportation:** Rental car/ride-sharing - $50
  - **Total:** $330

**Activities:**
- Check into your hotel a